In [5]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNetCV
import warnings
warnings.filterwarnings('ignore')
## グラフで日本語表示
import matplotlib as mpl
mpl.rcParams['font.family'] = 'IPAPGothic'
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6
rcParams["font.size"] = 18
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
import gc

pd.set_option("display.max_columns", 100)

In [12]:
df = pd.read_csv("pitch_data/pitch_gid_2018_03_01_milmlb_arimlb_1.csv")
df

,inning_num,top_bottom,pitcher_id,batter_id,batter_stand,pitch_type,start_speed,end_speed,spin_dir,spin_rate,pfx_x,pfx_z,px,pz,x0,y0,ax,ay,az,vx0,vy0,vz0,sz_top,sz_bot,break_y,break_angle,break_length,zone,nasty,cc,mt,type,code,event_num,des,tfs_zulu
0,1,top,643327,542979,R,SI,90.3,81.7,256.988,2019.502,-10.211094,2.359700,-0.913560,1.962431,-2.151212,50.0,-17.772380,27.431270,-28.066997,6.636647,-131.039012,-4.149642,3.481623,1.747950,23.8,29.7,7.7,13,69,NaN,NaN,S,C,3,Called Strike,2018-03-01T20:10:19Z
1,1,top,643327,542979,R,SI,90.4,82.6,256.725,2117.849,-10.668146,2.516827,0.741198,2.246606,-1.972296,50.0,-18.614222,26.668849,-27.782585,10.622379,-131.012707,-3.679357,3.448134,1.737258,23.8,30.2,7.7,14,51,NaN,NaN,S,C,4,Called Strike,2018-03-01T20:10:37Z
2,1,top,643327,542979,R,CH,81.1,75.5,226.279,546.789,-2.262741,2.163846,-0.212881,0.502041,-1.750697,50.0,-3.208909,20.462677,-29.105389,4.277438,-117.805165,-5.526370,3.428352,1.704245,23.9,4.9,8.6,13,16,NaN,NaN,B,B,5,Ball,2018-03-01T20:10:58Z
3,1,top,643327,542979,R,SI,90.9,83.9,248.989,1487.208,-7.071234,2.715983,0.365762,0.839602,-1.760256,50.0,-12.622162,24.577017,-27.326015,7.976950,-131.899992,-7.297680,3.451079,1.681508,23.9,21.5,7.0,14,32,NaN,NaN,B,B,6,Ball,2018-03-01T20:11:22Z
4,1,top,643327,542979,R,SI,90.4,83.3,233.645,1768.530,-7.395088,5.443001,0.921326,0.275552,-1.534116,50.0,-12.863932,27.533340,-22.705819,8.839219,-131.029206,-9.064738,3.428352,1.817926,23.8,24.4,6.3,14,48,NaN,NaN,B,B,7,Ball,2018-03-01T20:11:45Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,9,bottom,543118,605378,L,FF,91.4,84.7,159.380,1426.241,2.531600,6.727515,-1.182502,1.428004,-0.533349,50.0,4.608384,24.615213,-19.927652,-2.587769,-133.098853,-8.297065,3.413215,1.681919,23.9,-11.0,4.8,13,61,NaN,NaN,B,B,646,Ball,2018-03-01T23:06:58Z
293,9,bottom,543118,605378,L,FF,91.0,84.1,169.572,1443.696,1.327539,7.211863,1.173662,2.151377,-0.228580,50.0,2.388237,25.443459,-19.199941,3.232398,-132.567936,-6.559742,3.390488,1.681919,23.9,-8.3,4.6,14,49,NaN,NaN,S,F,647,Foul,2018-03-01T23:07:18Z
294,9,bottom,543118,605378,L,FS,85.5,78.1,213.731,870.253,-2.646149,3.962868,-0.563648,2.171961,-0.709177,50.0,-4.148380,25.418426,-25.961442,1.201788,-124.535428,-4.111653,3.413215,1.704657,23.8,8.3,7.0,7,33,NaN,NaN,X,X,648,"In play, out(s)",2018-03-01T23:07:46Z
295,9,bottom,543118,545121,L,FF,92.3,84.9,180.224,1935.258,-0.038492,9.755773,0.209448,1.524539,-0.131994,50.0,-0.070713,27.736091,-14.252081,0.920871,-134.354170,-8.768334,3.233845,1.467545,23.8,-0.4,3.5,8,52,NaN,NaN,S,C,653,Called Strike,2018-03-01T23:08:31Z


In [14]:
df["tfs_zulu"] = pd.to_datetime(df["tfs_zulu"])
df

,inning_num,top_bottom,pitcher_id,batter_id,batter_stand,pitch_type,start_speed,end_speed,spin_dir,spin_rate,pfx_x,pfx_z,px,pz,x0,y0,ax,ay,az,vx0,vy0,vz0,sz_top,sz_bot,break_y,break_angle,break_length,zone,nasty,cc,mt,type,code,event_num,des,tfs_zulu
0,1,top,643327,542979,R,SI,90.3,81.7,256.988,2019.502,-10.211094,2.359700,-0.913560,1.962431,-2.151212,50.0,-17.772380,27.431270,-28.066997,6.636647,-131.039012,-4.149642,3.481623,1.747950,23.8,29.7,7.7,13,69,NaN,NaN,S,C,3,Called Strike,2018-03-01 20:10:19+00:00
1,1,top,643327,542979,R,SI,90.4,82.6,256.725,2117.849,-10.668146,2.516827,0.741198,2.246606,-1.972296,50.0,-18.614222,26.668849,-27.782585,10.622379,-131.012707,-3.679357,3.448134,1.737258,23.8,30.2,7.7,14,51,NaN,NaN,S,C,4,Called Strike,2018-03-01 20:10:37+00:00
2,1,top,643327,542979,R,CH,81.1,75.5,226.279,546.789,-2.262741,2.163846,-0.212881,0.502041,-1.750697,50.0,-3.208909,20.462677,-29.105389,4.277438,-117.805165,-5.526370,3.428352,1.704245,23.9,4.9,8.6,13,16,NaN,NaN,B,B,5,Ball,2018-03-01 20:10:58+00:00
3,1,top,643327,542979,R,SI,90.9,83.9,248.989,1487.208,-7.071234,2.715983,0.365762,0.839602,-1.760256,50.0,-12.622162,24.577017,-27.326015,7.976950,-131.899992,-7.297680,3.451079,1.681508,23.9,21.5,7.0,14,32,NaN,NaN,B,B,6,Ball,2018-03-01 20:11:22+00:00
4,1,top,643327,542979,R,SI,90.4,83.3,233.645,1768.530,-7.395088,5.443001,0.921326,0.275552,-1.534116,50.0,-12.863932,27.533340,-22.705819,8.839219,-131.029206,-9.064738,3.428352,1.817926,23.8,24.4,6.3,14,48,NaN,NaN,B,B,7,Ball,2018-03-01 20:11:45+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,9,bottom,543118,605378,L,FF,91.4,84.7,159.380,1426.241,2.531600,6.727515,-1.182502,1.428004,-0.533349,50.0,4.608384,24.615213,-19.927652,-2.587769,-133.098853,-8.297065,3.413215,1.681919,23.9,-11.0,4.8,13,61,NaN,NaN,B,B,646,Ball,2018-03-01 23:06:58+00:00
293,9,bottom,543118,605378,L,FF,91.0,84.1,169.572,1443.696,1.327539,7.211863,1.173662,2.151377,-0.228580,50.0,2.388237,25.443459,-19.199941,3.232398,-132.567936,-6.559742,3.390488,1.681919,23.9,-8.3,4.6,14,49,NaN,NaN,S,F,647,Foul,2018-03-01 23:07:18+00:00
294,9,bottom,543118,605378,L,FS,85.5,78.1,213.731,870.253,-2.646149,3.962868,-0.563648,2.171961,-0.709177,50.0,-4.148380,25.418426,-25.961442,1.201788,-124.535428,-4.111653,3.413215,1.704657,23.8,8.3,7.0,7,33,NaN,NaN,X,X,648,"In play, out(s)",2018-03-01 23:07:46+00:00
295,9,bottom,543118,545121,L,FF,92.3,84.9,180.224,1935.258,-0.038492,9.755773,0.209448,1.524539,-0.131994,50.0,-0.070713,27.736091,-14.252081,0.920871,-134.354170,-8.768334,3.233845,1.467545,23.8,-0.4,3.5,8,52,NaN,NaN,S,C,653,Called Strike,2018-03-01 23:08:31+00:00


In [7]:
df["code"].unique()

array(['C', 'B', 'F', 'S', 'X', 'D', 'W', '*B', 'L', 'T', 'E'],
      dtype=object)

In [8]:
df[["code","des"]].drop_duplicates()

,code,des
0,C,Called Strike
2,B,Ball
5,F,Foul
6,S,Swinging Strike
11,X,"In play, out(s)"
14,D,"In play, no out"
71,W,Swinging Strike (Blocked)
79,*B,Ball In Dirt
92,L,Foul Bunt
145,T,Foul Tip
